In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

In [25]:
device = torch.device("xpu" if torch.xpu.is_available() else 'cpu')
print(f"Device : {device}")


Device : xpu


In [3]:
train = pd.read_csv("D:\\Python\\cnn\\fashion-mnist_train.csv")
test = pd.read_csv("D:\\Python\\cnn\\fashion-mnist_test.csv")
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train.shape

(60000, 785)

In [5]:
x_train , y_train = train.iloc[:,1:],train.iloc[:,0]
x_test , y_test = test.iloc[:,1:],test.iloc[:,0]

In [ ]:
# transformation for TRANSFER LEARNING  ( using VGG16)
"""
1) reshape to (28,28)
2) data type np.uint32
3) (3,28,28)
4) tensor has become PIL img
5) resize to (3,256,256)
6) centre crop to (3,224,224)
7) convert to tensor
8) normalize with mean and std
"""


In [10]:
# TRANSFROMATION
from torchvision import transforms
custom_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
from PIL import Image
import numpy as np

class CustomDataset(Dataset):
    def __init__(self,features,labels,transform):
        self.features = features
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.features)
    def __getitem__(self, idx):
        image = self.features[idx].reshape(28, 28).astype(np.uint8)
        image = np.stack([image]*3,axis=-1)
        image = Image.fromarray(image) # Convert to PIL Image
        image = self.transform(image)
        return image, torch.tensor(self.labels[idx], dtype=torch.long)


In [12]:
train_dataset = CustomDataset(x_train.values, y_train.values, transform=custom_transform)
test_dataset = CustomDataset(x_test.values, y_test.values, transform=custom_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [13]:
# fetch the pretrained model
import torchvision.models as models
vgg16 = models.vgg16(pretrained=True)

c:\Users\Lenovo\miniconda3\envs\torch_intel\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\miniconda3\envs\torch_intel\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\Lenovo/.cache\torch\hub\checkpoints\vgg16-397923af.pth


100%|██████████| 528M/528M [00:22<00:00, 24.9MB/s] 


In [16]:
vgg16.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [17]:
vgg16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [ ]:
# but to train the model on our dataset we need to have our own classifier

In [18]:
for param in vgg16.features.parameters():
    param.requires_grad = False # freeze the feature extractor


In [23]:
vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 10) 
)

In [26]:
vgg16 = vgg16.to(device)
lr = 0.001
epochs = 5

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=lr)

In [29]:
for i in range(epochs):
    total_loss = 0
    for batch_features, batch_labels in train_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        optimizer.zero_grad()
        outputs = vgg16(batch_features)
        loss = loss_fn(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    print(f"Epoch {i+1}/{epochs}, Loss: {total_loss/len(train_loader)}")

KeyboardInterrupt: 

In [ ]:
vgg16.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        outputs = vgg16(batch_features)
        _, predicted = torch.max(outputs.data, 1)

        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()
    print(f"Test Accuracy: {100 * correct / total:.2f}%")
